# Tarea 5
### Sebastián Murillo García 191838
15 de noviembre, 2021

---

**Nota**: Se dejaron comentadas aquellas "impresiones" de los *Dataframes* que se fueron generando en el notebook durante la resolución de la tarea, pero que no responde a ninguna pregunta de la tarea. Simplemente se tienen para tener un control más claro de lo que estamos haciendo en cada celda de código.

In [25]:
import pandas as pd

### 1. Carga de datos a Pandas

---


*    Utilizamos `pd.read_csv` para leer y almacenar en un *dataframe* el archivo `.csv` correspondiente a `casos_nacionales_covid-19`.

In [26]:
casos_covid = pd.read_csv("https://archivo.datos.cdmx.gob.mx/casos_nacionales_covid-19.csv")

*    Eliminamos la primera columna (`Unnamed: 0`) con la función `drop` sin crear un nuevo *dataframe*.

In [27]:
casos_covid.drop(['Unnamed: 0'], axis=1, inplace=True)

*    Cambiamos los tipos de datos de las columnas `fecha_ingreso`, `fecha_sintomas` y `fecha_def` a `date` con la función `to_datetime()` de pandas. El formato de la fecha está como `yyyy-mm-dd`.

In [ ]:
casos_covid.fecha_ingreso = pd.to_datetime(casos_covid.fecha_ingreso, errors='coerce', format='%Y-%m-%d')
casos_covid.fecha_sintomas = pd.to_datetime(casos_covid.fecha_sintomas, errors='coerce', format='%Y-%m-%d')
casos_covid.fecha_def = pd.to_datetime(casos_covid.fecha_def, errors='coerce', format='%Y-%m-%d')

#casos_covid

### 2. Análisis de defunciones

---

*    Generamos un *dataframe* llamado `defunciones`, el cual solamente contiene registros que tienen asociada una fecha de defunción.

In [55]:
defunciones = casos_covid[casos_covid.fecha_def.notnull()]
#defunciones

*    Conteo de registros existentes por entidad de residencia (`entidad_res`).

In [59]:
num_defunciones_x_entidad = defunciones.groupby(['entidad_res'], as_index=False)['id_registro'].count().rename(columns={'id_registro': 'num_defunciones'})
num_defunciones_x_entidad

,entidad_res,num_defunciones
0,GUANAJUATO,62
1,GUERRERO,193
2,HIDALGO,528
3,JALISCO,13
4,MICHOACÁN DE OCAMPO,81
5,MORELOS,179
6,MÉXICO,16662
7,NAYARIT,3
8,NUEVO LEÓN,14
9,OAXACA,89


*    Obtención del top 10 de entidades con mayor número de defunciones.

In [61]:
top_10_num_def_x_entidad = num_defunciones_x_entidad.sort_values(by=['num_defunciones'], ascending=False).head(10)
top_10_num_def_x_entidad

,entidad_res,num_defunciones
6,MÉXICO,16662
2,HIDALGO,528
10,PUEBLA,298
1,GUERRERO,193
5,MORELOS,179
19,VERACRUZ DE IGNACIO DE LA LLAVE,117
9,OAXACA,89
4,MICHOACÁN DE OCAMPO,81
18,TLAXCALA,72
0,GUANAJUATO,62


### 3. Análisis de comorbilidad

---

*    Se acomodaron los datos para que las columnas: `neumonia`, `embarazo`, `diabetes`, `epoc`, `asma`, `inmusupr`, `hipertension`, `otra_com`, `cardiovascular`, `obesidad`, `renal_cronica`, `tabaquismo` y `otro_caso`, estuvieran en una sola columna llamada `comorbilidad` y el valor asociado estuviera en otra llamada `valor`.

In [62]:
comorbilidad_def = defunciones.melt(id_vars=['id_registro','entidad_res'], 
                 value_vars=['neumonia','embarazo','diabetes','epoc','asma','inmusupr','hipertension','otra_com','cardiovascular','obesidad','renal_cronica','tabaquismo','otro_caso'], 
                 var_name="comorbilidad", value_name="valor")
#comorbilidad_def

*    Conteo de registros de defunciones existentes por `comorbilidad`.

In [63]:
num_def_x_comorbilidad = comorbilidad_def[comorbilidad_def['valor'] == 'SI'].groupby(['comorbilidad'], as_index=False)['id_registro'].count().rename(columns={'id_registro': 'num_defunciones'})
num_def_x_comorbilidad

,comorbilidad,num_defunciones
0,asma,873
1,cardiovascular,3576
2,diabetes,23121
3,embarazo,40
4,epoc,2962
5,hipertension,26990
6,inmusupr,2001
7,neumonia,50683
8,obesidad,13398
9,otra_com,3649


*    Conteo de registros de defunciones existentes por entidad de residencia (`entidad_res`) y `comorbilidad`.

In [64]:
num_def_x_comorbilidad_res = comorbilidad_def[comorbilidad_def['valor'] == 'SI'].groupby(['entidad_res','comorbilidad'], as_index=False)['id_registro'].count().rename(columns={'id_registro': 'num_defunciones'})
num_def_x_comorbilidad_res

,entidad_res,comorbilidad,num_defunciones
0,GUANAJUATO,asma,1
1,GUANAJUATO,cardiovascular,3
2,GUANAJUATO,diabetes,17
3,GUANAJUATO,epoc,1
4,GUANAJUATO,hipertension,14
...,...,...,...
203,ZACATECAS,asma,1
204,ZACATECAS,diabetes,1
205,ZACATECAS,epoc,1
206,ZACATECAS,neumonia,3


### 4. Casos de sobrevivientes

---

*Dataframe* de sobrevivientes:

In [35]:
sobrevivientes = casos_covid[casos_covid.fecha_def.isnull()]
#sobrevivientes

*    Porcentaje de registros de sobrevivientes que estuvieron entubados y no entubados (o cualquier otro caso).

In [65]:
porcentaje_intubados = sobrevivientes.value_counts(['intubado'], normalize=True).apply(lambda x: x * 100)
porcentaje_intubados

intubado       
NO APLICA          96.876159
NO                  2.941100
SI                  0.167363
NO ESPECIFICADO     0.015377
dtype: float64

*Dataframe* con columna `comorbilidad` para el caso de los sobrevivientes:

In [ ]:
comorbilidad_sobrev = sobrevivientes.melt(id_vars=['id_registro','entidad_res'], 
                 value_vars=['neumonia','embarazo','diabetes','epoc','asma','inmusupr','hipertension','otra_com','cardiovascular','obesidad','renal_cronica','tabaquismo','otro_caso'], 
                 var_name="comorbilidad", value_name="valor")
#comorbilidad_sobrev

*    Conteo de registros de sobrevivientes existentes por entidad de residencia (`entidad_res`) y `comorbilidad`.

In [66]:
num_reg_x_comorbilidad_res = comorbilidad_sobrev[comorbilidad_sobrev['valor'] == 'SI'].groupby(['entidad_res','comorbilidad'], as_index=False)['id_registro'].count().rename(columns={'id_registro': 'num_registros'})
num_reg_x_comorbilidad_res

,entidad_res,comorbilidad,num_registros
0,DURANGO,asma,6
1,DURANGO,cardiovascular,2
2,DURANGO,diabetes,11
3,DURANGO,epoc,2
4,DURANGO,hipertension,16
...,...,...,...
283,ZACATECAS,neumonia,10
284,ZACATECAS,obesidad,6
285,ZACATECAS,otra_com,5
286,ZACATECAS,otro_caso,61


### 5. Análisis estadísticos

---

Generamos una columna llamada `dias_hospitalizado` que obtiene a los registros de defunciones donde el tipo de paciente es `HOSPITALIZADO`, para hacer las siguientes estadísticas con su correspondiente interpretación:

In [39]:
defs_hospitalizado = defunciones[defunciones.tipo_paciente == "HOSPITALIZADO"]
#defs_hospitalizado

In [41]:
dias_hospitalizado = defs_hospitalizado.fecha_def-defs_hospitalizado.fecha_ingreso
defs_hospitalizado.insert(loc=13, column="dias_hospitalizado", value=dias_hospitalizado)
# defs_hospitalizado

*    Min

In [42]:
defs_hospitalizado.dias_hospitalizado.min()

Timedelta('-365 days +00:00:00')

**Interpretación**: El número de días de la persona que estuvo menos días hospitalizado antes de fallecer es de **-365 días**. ¿Es posible esto? No, en este momento nos percatamos que es posible que existan erorres en los registros. Los trataremos más adelante.

* Max

In [43]:
defs_hospitalizado.dias_hospitalizado.max()

Timedelta('430 days 00:00:00')

**Interpretación**: El número de días de la persona que estuvo más días hospitalizado antes de fallecer es de **430 días**.

* Mediana

In [44]:
defs_hospitalizado.dias_hospitalizado.median()

Timedelta('6 days 00:00:00')

**Interpretación**: La mitad de los pacientes que fallecieron estuvieron hospitalizados por debajo de 6 días, mientras que la otra mitad de los pacientes que fallecieron estuvieron por arriba de los 6 días.

* Promedio

In [45]:
defs_hospitalizado.dias_hospitalizado.mean()

Timedelta('9 days 00:20:43.280140208')

**Interpretación**: Las personas que fallecieron por Covid-19 estuvieron hospitalizados aproximadamente 9 días.

* Desviación Estándar

In [46]:
defs_hospitalizado.dias_hospitalizado.std()

Timedelta('11 days 13:50:37.822931424')

**Interpretación**: Las personas que fallecieron por Covid-19 estuvieron hospitalizados aproximadamente entre -2 días y 20 días. De nuevo, el primer dato del rango no es posible.

* Primer Cuartil

In [47]:
defs_hospitalizado.dias_hospitalizado.quantile(0.25)

Timedelta('3 days 00:00:00')

**Interpretación**: El 25% de las personas que fallecieron por covid estuvieron hospitalizadas, máximo, hasta por 3 días.

* Tercer Cuartil

In [48]:
defs_hospitalizado.dias_hospitalizado.quantile(0.75)

Timedelta('12 days 00:00:00')

**Interpretación**: El 75% de las personas que fallecieron por covid estuvieron hospitalizadas, máximo, hasta por 12 días.

### Análisis extra.

*Dataframe* (y conteo) de los registros que tienen un error. ¿Cuál error? La fecha de ingreso al hospital es más reciente que la fecha de defunción.

In [49]:
registros_error = defs_hospitalizado[defs_hospitalizado.dias_hospitalizado < pd.Timedelta(days=0)]
num_errores = registros_error.shape[0]

print("El número de registros que tienen errores en la fecha de ingreso y/o fecha de defunción es: "+str(num_errores))

El número de registros que tienen errores en la fecha de ingreso y/o fecha de defunción es: 45


*Dataframe* con el filtrado de los registros que tienen la fecha de defunción más reciente que la fecha de ingreso al hospital.

In [50]:
registros_dias_positivos = defs_hospitalizado[defs_hospitalizado.dias_hospitalizado >= pd.Timedelta(days=0)]
#registros_dias_positivos

In [51]:
# Obtención de datos
min = registros_dias_positivos.dias_hospitalizado.min()
max = registros_dias_positivos.dias_hospitalizado.max()
mediana = registros_dias_positivos.dias_hospitalizado.median()
promedio = registros_dias_positivos.dias_hospitalizado.mean()
desv_std = registros_dias_positivos.dias_hospitalizado.std()
q1 = registros_dias_positivos.dias_hospitalizado.quantile(0.25)
q3 = registros_dias_positivos.dias_hospitalizado.quantile(0.75)

# Impresión de datos
print("Mínimo: {}".format(min))
print("Máximo: {}".format(max))
print("Mediana: {}".format(mediana))
print("Promedio: {}".format(promedio))
print("Desviación estándar: {}".format(desv_std))
print("Primer cuartil: {}".format(q1))
print("Tercer cuartil: {}".format(q3))


Mínimo: 0 days 00:00:00
Máximo: 430 days 00:00:00
Mediana: 6 days 00:00:00
Promedio: 9 days 00:45:28.868786634
Desviación estándar: 11 days 11:27:12.996688699
Primer cuartil: 3 days 00:00:00
Tercer cuartil: 12 days 00:00:00


**Interpretación**: Es análoga a la realizada en la pregunta 5. Debido al tiempo, no pude terminar este análisis. Lo siento, maestra. Le he fallado.